# TEST code

fixed number to stochastic conversion & multiply & stochastic to fixed number conversion

## Import

In [1]:
import random, struct, math
import torch
import numpy as np
import torch.nn as nn 
import argparse

## Parser

In [2]:
parser = argparse.ArgumentParser(description='fixed_mac')
parser.add_argument('--full_bits', type=int, default=8, help='Number of Quantization Bits')
parser.add_argument('--frac_bits', type=int, default=6, help='Number of Quantization Bits')
parser.add_argument('--bBW', type=int, default=4, help='Number of bit width')
args = parser.parse_args(args=[])

## Definition

### int2bin : int number to binary number

In [3]:
def int2bin(iIn,iBW):
    iBW = iBW + 1
    if iIn >= 0:
        bOut = bin(iIn).replace('0b','').rjust(iBW,'0')
    else :
        bOut = bin(iIn & (pow(2,iBW)-1)).replace('0b','').rjust(iBW,'1')
    return bOut[1:]

### XOR : xor gate with string

In [4]:
def XOR(iA,iB):
    if iA != iB :
        iOut = '1'
    else : 
        iOut = '0'
    return iOut

### SNUM : sign number determination

In [5]:
def snum(a):
    if a > 0 :
        return '0'
    else :
        return '1'

### fxp & flp2fix : floating number to fixed number conversion

In [6]:
class fxp:
    def __init__(self, bIn, iBWF):
        self.iFullBW = len(bIn)
        self.iIntgBW = self.iFullBW - iBWF
        self.bSign = bIn[0]
        self.bIntg = bIn[:self.iIntgBW]
        self.bFrac = bIn[self.iIntgBW:]
        self.fFull = 0
        try:
            for idx, bit in enumerate(bIn):
                if idx == 0:
                    self.fFull = self.fFull + int(bit,2) * -pow(2, self.iIntgBW - 1)
                else:
                    self.fFull = self.fFull + int(bit,2) * pow(2, self.iIntgBW - 1 - idx)
        except:
            print(bIn)
        self.dispFull = self.bIntg +"."+ self.bFrac 
        return

In [7]:
class flp2fix:
    def __init__(self, fIn, iBW, iBWF):
        self.fMin = - 2 ** (iBW - iBWF - 1)
        self.fMax = (2 ** (iBW-1) - 1) * (2 ** -iBWF)
        self.fResol = 2 ** -iBWF
        if fIn < self.fMin or fIn > self.fMax:
            print(f'({fIn}): Out of input range ({self.fMax}/{self.fMin}) during flp -> fix converting ')
        self.iBW = iBW
        self.iBWI = iBW - iBWF
        self.iBWF = iBWF

        self.iFLP2INT = abs(int(fIn * 2 ** iBWF))
        if fIn < 0:
            self.iFLP2INT = 2 ** (iBW-1) - self.iFLP2INT

        if fIn >= 0:
            self.bFull = bin(self.iFLP2INT)[2:].rjust(iBW, '0')
        else:
            self.bFull = '1'+bin(self.iFLP2INT)[2:].rjust(iBW-1, '0')
            if len(self.bFull) > iBW:
                self.bFull = '0' * iBW

        self.cssFxp = fxp(self.bFull, self.iBWF)
        self.bSign = self.cssFxp.bSign
        self.bIntg = self.cssFxp.bIntg
        self.bFrac = self.cssFxp.bFrac
        self.fFull = self.cssFxp.fFull
        return

### LFSR : make pseudorandom number bitstream

In [8]:
class LFSR:
    def Random(self):
        self.b0 = eval(f'str(random.randint(0,1))')
        self.b1 = eval(f'str(random.randint(0,1))')
        self.b2 = eval(f'str(random.randint(0,1))')
        self.b3 = eval(f'str(random.randint(0,1))')
        
        return self.b0 + self.b1 + self.b2 + self.b3
    
    def Normal(self,stream):
        self.b0 = XOR(int(stream[2]),int(stream[3]))
        self.b1 = stream[0]
        self.b2 = stream[1]
        self.b3 = stream[2]
        
        return self.b0 + self.b1 + self.b2 + self.b3
    
    def Allzero(self):
        self.b0 = '0'
        self.b1 = '0'
        self.b2 = '0'
        self.b3 = '0'
        
        return self.b0 + self.b1 + self.b2 + self.b3

### LFSRlist : make pseudorandom number bitstream with 2**bBW cycle

In [9]:
def LFSRlist():
    lfsr = LFSR()
    lfsrlist = []
    for k in range(2**(args.bBW)-1): #lfsr number generating
        if k == 0:
            lfsrlist.append(lfsr.Random())
        else :
            lfsrlist.append(lfsr.Normal(lfsrlist[k-1]))
        if (k == 2**(args.bBW)-2):
            lfsrlist.append(lfsr.Allzero())
    
    if (args.bBW) != args.frac_bits :
        if args.bBW < args.frac_bits :
            for i in range(len(lfsrlist)):
                lfsrlist[i] = lfsrlist[i] + (args.frac_bits-args.bBW)*'0'
        else :
            print("it can't work")
            return 0
    
    return lfsrlist

### Comp : Comparator in SNG

In [10]:
def Comp(a,lfsr,snum):
    for com in range(0,len(a)):
        oA = '0'
        if a[com]!=lfsr[com]:
            if(int(a[com]) > int(lfsr[com])):
                oA = '1'
            break
    return XOR(oA,snum)

### Perm : module for permutation SNG 

In [11]:
def perm(a):
    al = len(a)
    blist = []
    for i in range(al) :
        #print(al-i-1)
        blist.append(a[al-i-1])
    
    b = "".join(blist)
    
    return b

### findActMaxMin : find abs max value in act tensor

In [12]:
def findActMaxMin():
    SF = {}
    for i in range(3):
            max = torch.max(eval(f'act{i}'))
            min = torch.min(eval(f'act{i}'))
            exec(f"SF['act{i}']=torch.max(abs(max),abs(min)).item()")
    return SF

### findWeiMaxMin : find abs max balue in weight tensor --> it will be changed in MLP module

In [13]:
def findWeiMaxMin():
    SF = {}
    for i in range(3):
            max = torch.max(eval(f'wei{i}'))
            min = torch.min(eval(f'wei{i}'))
            exec(f"SF['wei{i}']=torch.max(abs(max),abs(min)).item()")
    return SF

### SNG : stochastic number generator module

In [14]:
def SNG(iIN,lfsr):

    sNUM = snum(iIN)
    
    bIN = flp2fix(iIN,args.full_bits,args.frac_bits).bFull
    oAlist = []
    
    for k in range(2**(args.bBW)): #lfsr number generating
        lNUM = lfsr[k]
        oAlist.append(Comp(bIN[-(args.frac_bits):],lNUM,sNUM)) #comparator of input a
        
    oAnum = oAlist.count('1')
    
    oAlist.insert(0,sNUM)
    sA = "".join(oAlist)
    return sA

### SNG_P : permutation stochastic number generator module

In [15]:
def SNG_P(iIN,lfsr):
    sNUM = snum(iIN)
    
    bIN = flp2fix(iIN,args.full_bits,args.frac_bits).bFull
    oAlist = []
    
    for k in range(2**(args.bBW)): #lfsr number generating
        if (args.bBW == args.frac_bits) :    
            lNUM = perm(lfsr[k])
        elif (args.bBW < args.frac_bits) :
            lNUM = perm(lfsr[k][:args.bBW])+(args.frac_bits-args.bBW)*"0"
        oAlist.append(Comp(bIN[-(args.frac_bits):],lNUM,sNUM)) #comparator of input a
        
    oAnum = oAlist.count('1')
    
    oAlist.insert(0,sNUM)
    sA = "".join(oAlist)
    return sA

### SNGnumpy : SNG module with numpy

In [16]:
def SNGnumpy(fIn,lfsr):
    sList = []
    for aNumpy in fIn.view(-1):
        sList.append(SNG(float(aNumpy),lfsr))
                     
    return np.array(sList).reshape(fIn.size())

### SNGpnumpy : permutation SNG module with numpy

In [17]:
def SNGpnumpy(fIn,lfsr):
    sList = []
    for aNumpy in fIn.view(-1):
        sList.append(SNG_P(float(aNumpy),lfsr))
                     
    return  np.array(sList).reshape(fIn.size())

### CountOne : count 1 in stochastic number bit stream

In [18]:
def CountOne(nIn):
    nlist = []
    for num in nIn.reshape(-1):
        n = 0
        for a in num:
            if a == '1' :
                n += 1
        if a[0] == '1' :
            nlist.append(n-1)
        else :
            nlist.append(n)
    return torch.tensor(nlist).view(nIn.shape)

### defSign : sign determination in S2N 

In [19]:
def defSign(nIn):
    nlist = []
    for num in nIn.reshape(-1):
        if num[0] == '1' :
            nlist.append(-1)
        else :
            nlist.append(1)
    return torch.tensor(nlist).view(nIn.shape)

## test

In [20]:
act0 = torch.rand((9,9))-torch.randint(4,(9,9))+torch.randint(4,(9,9))
act1 = torch.rand((9,9))-torch.randint(4,(9,9))+torch.randint(4,(9,9))
act2 = torch.rand((9,9))-torch.randint(4,(9,9))+torch.randint(4,(9,9))

wei0 = torch.rand((9,9))-torch.randint(4,(9,9))+torch.randint(4,(9,9))
wei1 = torch.rand((9,9))-torch.randint(4,(9,9))+torch.randint(4,(9,9))
wei2 = torch.rand((9,9))-torch.randint(4,(9,9))+torch.randint(4,(9,9))

### number to stochastic conversion

In [21]:
def N2S(act0,act1,act2,wei0,wei1,wei2):
    aSF = findActMaxMin()
    wSF = findWeiMaxMin()
    
    lfsrlist0 = LFSRlist()
    lfsrlist1 = LFSRlist()
    lfsrlist2 = LFSRlist()
    
    a0 = SNGnumpy(act0/aSF['act0'],lfsrlist0)
    a1 = SNGnumpy(act1/aSF['act1'],lfsrlist1)
    a2 = SNGnumpy(act2/aSF['act2'],lfsrlist2)
    
    w0 = SNGpnumpy(wei0/wSF['wei0'],lfsrlist0)
    w1 = SNGpnumpy(wei1/wSF['wei1'],lfsrlist1)
    w2 = SNGpnumpy(wei2/wSF['wei2'],lfsrlist2)
    
    return a0,a1,a2,w0,w1,w2

### stochastic to number conversion

In [22]:
def S2N(a0,a1,a2,w0,w1,w2):
    aSF = findActMaxMin()
    wSF = findWeiMaxMin()
    
    sa0 = defSign(a0)
    sa1 = defSign(a1)
    sa2 = defSign(a2)
    sw0 = defSign(w0)
    sw1 = defSign(w1)
    sw2 = defSign(w2)
    

    ca0 = CountOne(a0)/(2**args.bBW) * aSF['act0'] *sa0
    ca1 = CountOne(a1)/(2**args.bBW) * aSF['act1'] *sa1
    ca2 = CountOne(a2)/(2**args.bBW) * aSF['act2'] *sa2
    cw0 = CountOne(w0)/(2**args.bBW) * wSF['wei0'] *sw0
    cw1 = CountOne(w1)/(2**args.bBW) * wSF['wei1'] *sw1
    cw2 = CountOne(w2)/(2**args.bBW) * wSF['wei2'] *sw2

    return ca0,ca1,ca2,cw0,cw1,cw2

In [23]:
a0, a1, a2, w0, w1, w2 = N2S(act0,act1,act2,wei0,wei1,wei2)

In [24]:
print(w0)

[['00000000001100101' '00010000001110111' '00000000000000001'
  '00000000001100001' '00000000001000001' '00000000000000001'
  '10000011000000000' '00010000001110111' '11101011100000000']
 ['00000000001110101' '11000011100000000' '00010100011111111'
  '00000000001100101' '11101011110001000' '00010000001110111'
  '00000000001100001' '00010000001110111' '00010000001110101']
 ['00010100011111111' '11111111111111111' '00010100001110111'
  '00010100001110111' '00000000001100001' '01111101111111111'
  '00111100111111111' '00111100111111111' '00111100111111111']
 ['00000000000000001' '00010000001110101' '00010000001110101'
  '11111111110011010' '00110100011111111' '00000000001100101'
  '10000011000000000' '11111111111111111' '00010000001110111']
 ['00010000001110111' '11001011100000000' '10000010000000000'
  '00010000001110101' '00000000001100001' '00000000000000000'
  '01111100111111111' '00010000001110111' '00010100001110111']
 ['11101111110001010' '10000000000000000' '00010100011110111'
  '

In [25]:
print(wei0)

tensor([[ 0.8780,  1.6413,  0.0834,  0.6942,  0.4752,  0.0841, -0.6249,  1.5531,
         -1.6376],
        [ 1.0854, -1.0785,  2.4155,  0.8456, -2.0495,  1.6730,  0.6613,  1.7166,
          1.4565],
        [ 2.4452, -0.0105,  1.9659,  1.8008,  0.7593,  3.5958,  2.9718,  3.0541,
          3.1196],
        [ 0.2344,  1.3016,  1.3671, -2.8875,  2.6560,  0.9278, -0.6146, -0.0318,
          1.5361],
        [ 1.5415, -1.2582, -0.2944,  1.3481,  0.5706,  3.8392,  3.3007,  1.7022,
          1.7985],
        [-2.5664, -0.0622,  1.9809,  0.8991,  0.4824,  2.5708,  1.2358, -0.4540,
          0.7896],
        [ 1.7822,  0.0802,  2.4515,  3.7230, -0.8989,  1.8529, -2.0271, -2.8099,
          3.3923],
        [-1.2149,  3.1959,  0.1934, -0.3689, -1.6635,  1.8168, -1.6835,  2.6355,
         -1.2184],
        [ 0.9208, -0.1225, -0.9702,  2.9331,  3.3130, -2.6868,  0.0886,  0.3429,
          1.3955]])


In [26]:
ca0, ca1, ca2, cw0, cw1, cw2 = S2N(a0,a1,a2,w0,w1,w2)

In [27]:
def FindError(a,b):
    error = 0
    for anum,bnum in zip(a.view(-1),b.view(-1)):
        e = (abs(bnum-anum)*100).item()
        error += e
    return error/(len(a.view(-1))+len(b.view(-1)))

In [28]:
ea0 = FindError(act0,ca0)
ea1 = FindError(act1,ca1)
ea2 = FindError(act2,ca2)
ew0 = FindError(wei0,cw0)
ew1 = FindError(wei1,cw1)
ew2 = FindError(wei2,cw2)

In [30]:
print(f'act0 error is {ea0}')
print(f'act1 error is {ea1}')
print(f'act2 error is {ea2}')
print(f'wei0 error is {ew0}')
print(f'wei1 error is {ew1}')
print(f'wei2 error is {ew2}')

act0 error is 9.938985936435653
act1 error is 10.016247471173605
act2 error is 12.874776237908705
wei0 error is 15.080154692685163
wei1 error is 9.629665031845187
wei2 error is 9.965427566457677
